In [289]:
# Python imports

import os
import pandas as pd
import time
import logging
import json

# Module logger
log = logging.getLogger(__name__)

# For timing the run
start = time.time()

import numpy as np
import matplotlib.pyplot as plt
import oggm
from oggm import cfg, utils, workflow, tasks, graphics
from oggm.workflow import execute_entity_task, gis_prepro_tasks, climate_tasks

# Initialize OGGM and set up the run parameters --> Read the configuration file containing the run’s parameters.
cfg.initialize(logging_level='WORKFLOW')

2019-06-26 17:05:43: oggm.cfg: Using configuration file: /home/pelto/anaconda2/envs/oggm_env/oggm/oggm/params.cfg


In [290]:
# Pre-download other files which will be needed later
# utils.get_cru_cl_file()
# utils.get_cru_file(var='tmp')
# utils.get_cru_file(var='pre')

# Get the RGI glaciers for the run.
rgi_list = ['RGI60-02.07780', 'RGI60-02.01857', 'RGI60-02.02171', 'RGI60-02.04264', 'RGI60-02.03688', 'RGI60-02.00147']
rgidf = utils.get_rgi_glacier_entities(rgi_list)

In [291]:
cfg.PARAMS['border'] = 10
cfg.PARAMS['grid_dx_method']
cfg.PARAMS['run_mb_calibration'] = True

In [294]:
WORKING_DIR = '/home/pelto/oggm_runs/CB_6_mbcal_gradient' 
cfg.PATHS['working_dir'] = WORKING_DIR

gdirs = workflow.init_glacier_regions(rgidf, reset=True, force=True, from_prepro_level=3) 
#from_prepro_level=3, prepro will override cfg.params --at least for fixed_dx

gis_prepro_tasks(gdirs)

# climate_tasks(gdirs) ## fails with TypeError: zip argument #2 must support iteration

2019-06-26 17:06:57: oggm.workflow: init_glacier_regions from prepro level 3 on 6 glaciers.
2019-06-26 17:06:57: oggm.workflow: Execute entity task gdir_from_prepro on 6 glaciers
2019-06-26 17:06:57: oggm.workflow: Multiprocessing: using all available processors (N=8)
2019-06-26 17:07:21: oggm.workflow: Execute entity task glacier_masks on 6 glaciers
2019-06-26 17:07:21: oggm.workflow: Execute entity task compute_centerlines on 6 glaciers
2019-06-26 17:07:21: oggm.workflow: Execute entity task initialize_flowlines on 6 glaciers
2019-06-26 17:07:21: oggm.workflow: Execute entity task compute_downstream_line on 6 glaciers
2019-06-26 17:07:21: oggm.workflow: Execute entity task compute_downstream_bedshape on 6 glaciers
2019-06-26 17:07:21: oggm.workflow: Execute entity task catchment_area on 6 glaciers
2019-06-26 17:07:22: oggm.workflow: Execute entity task catchment_intersections on 6 glaciers
/home/pelto/anaconda2/envs/oggm_env/lib/python3.6/site-packages/geopandas/io/file.py:108: Fiona

In [295]:
from oggm.core.climate import apparent_mb_from_linear_mb

#re-run the calibration
cfg.PARAMS['run_mb_calibration'] = True
# tasks.apparent_mb_from_linear_mb(gdirs)  #, mb_gradient=5.8)

#later can try MB gradient specific to each glacier
workflow.execute_entity_task(tasks.apparent_mb_from_linear_mb, gdirs, mb_gradient=5.8)

2019-06-26 17:07:23: oggm.workflow: Execute entity task apparent_mb_from_linear_mb on 6 glaciers
2019-06-26 17:07:23: oggm.workflow: Multiprocessing: using all available processors (N=8)


[None, None, None, None, None, None]

In [296]:
from oggm.core import massbalance
from oggm.core.massbalance import LinearMassBalance
# from oggm.core.flowline import FluxBasedModel

In [297]:
# oggm-edu gradients example uses get_annual_mb, I used get_specific_mb because 
#     it was the only recognized option

##maybe I can't just define these parameters here...the model must already have these parameters, 
#     thus I just need to define them here so that they are recognized 
nx = 100
# bed_h = np.linspace(3400, 1400, nx)
surface_h = np.linspace(3300, 1800, nx) #bed_h

for i in range(6):
    annual_mb = []
mb_gradient = 5.8

for i in range(6):
    gdir = gdirs[i]
#     mbmod = tasks.apparent_mb_from_linear_mb(gdir)
    mbmod = massbalance.LinearMassBalance(gdir) #check_calib_params=False)
    a = LinearMassBalance(ela_h=[0], grad=mb_gradient)
        ##what does ela_h=[0] imply??
        
    # Calculation of the annual mass balance along the glacier profile
    annual_mb.append(a.get_annual_mb(surface_h) * cfg.SEC_IN_YEAR)

In [298]:
# Inversion tasks
execute_entity_task(tasks.prepare_for_inversion, gdirs)
# We use the default parameters for this run
execute_entity_task(tasks.mass_conservation_inversion, gdirs)
execute_entity_task(tasks.filter_inversion_output, gdirs)
    
# Compile output
log.info('Compiling output')
utils.compile_glacier_statistics(gdirs)

2019-06-26 17:16:28: oggm.workflow: Execute entity task prepare_for_inversion on 6 glaciers
2019-06-26 17:16:28: oggm.workflow: Execute entity task mass_conservation_inversion on 6 glaciers
2019-06-26 17:16:28: oggm.workflow: Execute entity task filter_inversion_output on 6 glaciers
2019-06-26 17:16:28: oggm.workflow: Execute entity task glacier_statistics on 6 glaciers


,rgi_region,rgi_subregion,name,cenlon,cenlat,rgi_area_km2,glacier_type,terminus_type,status,inv_volume_km3,...,flowline_mean_elev,flowline_max_elev,flowline_min_elev,flowline_avg_width,flowline_avg_slope,t_star,mu_star_glacierwide,mu_star_flowline_avg,mu_star_allsame,mb_bias
rgi_id,,,,,,,,,,,,,,,,,,,,,
RGI60-02.00147,02,02-03,,-117.137,49.7490,1.801,Glacier,Land-terminating,Glacier or ice cap,0.100547,...,2582.918205,2746.576169,2268.645356,1350.074963,0.254657,1953,107.210031,107.210031,True,217.414414
RGI60-02.01857,02,02-03,,-115.301,50.7108,2.585,Glacier,Land-terminating,Glacier or ice cap,0.267864,...,2670.453796,2790.921187,2486.550776,1151.960784,0.152125,1955,48.643157,48.643157,True,287.547872
RGI60-02.02171,02,02-03,,-116.930,50.8069,16.898,Glacier,Land-terminating,Glacier or ice cap,1.849588,...,2547.654439,3151.528725,1877.053045,2004.032258,0.190631,1946,110.274153,110.305877,False,298.550866
RGI60-02.03688,02,02-03,,-117.434,51.2370,5.944,Glacier,Land-terminating,Glacier or ice cap,0.584304,...,2497.830423,2830.994833,2039.689477,1929.870130,0.165791,1937,160.034335,160.034335,True,321.314838
RGI60-02.04264,02,02-03,,-117.710,51.4292,4.425,Glacier,Land-terminating,Glacier or ice cap,0.261162,...,2495.373679,2858.329086,2013.129297,1990.553306,0.306335,1935,151.840873,151.864583,False,320.478366
RGI60-02.07780,02,02-03,,-119.581,52.6705,6.490,Glacier,Land-terminating,Glacier or ice cap,0.496243,...,2337.094173,2777.189619,1738.905177,1550.406116,0.268959,1946,63.827941,63.827941,True,350.451336


In [299]:
utils.compile_glacier_statistics(gdirs, inversion_only=True)

2019-06-26 17:16:36: oggm.workflow: Execute entity task glacier_statistics on 6 glaciers


,rgi_region,rgi_subregion,name,cenlon,cenlat,rgi_area_km2,glacier_type,terminus_type,status,inv_volume_km3,inv_thickness_m,vas_volume_km3,vas_thickness_m,dem_source,n_orig_centerlines,perc_invalid_flowline
rgi_id,,,,,,,,,,,,,,,,
RGI60-02.00147,02,02-03,,-117.137,49.7490,1.801,Glacier,Land-terminating,Glacier or ice cap,0.100547,55.828529,0.076350,42.393235,SRTM,2,0.0
RGI60-02.01857,02,02-03,,-115.301,50.7108,2.585,Glacier,Land-terminating,Glacier or ice cap,0.267864,103.622316,0.125491,48.545796,SRTM,4,0.0
RGI60-02.02171,02,02-03,,-116.930,50.8069,16.898,Glacier,Land-terminating,Glacier or ice cap,1.849588,109.456043,1.658641,98.156069,SRTM,5,0.0
RGI60-02.03688,02,02-03,,-117.434,51.2370,5.944,Glacier,Land-terminating,Glacier or ice cap,0.584304,98.301548,0.394310,66.337403,SRTM,2,0.0
RGI60-02.04264,02,02-03,,-117.710,51.4292,4.425,Glacier,Land-terminating,Glacier or ice cap,0.261162,59.019626,0.262791,59.387685,SRTM,3,0.0
RGI60-02.07780,02,02-03,,-119.581,52.6705,6.490,Glacier,Land-terminating,Glacier or ice cap,0.496243,76.462757,0.444954,68.559979,SRTM,4,0.0


In [300]:
annual_mb

[array([21.26666667, 21.16902357, 21.07138047, 20.97373737, 20.87609428,
        20.77845118, 20.68080808, 20.58316498, 20.48552189, 20.38787879,
        20.29023569, 20.19259259, 20.09494949, 19.9973064 , 19.8996633 ,
        19.8020202 , 19.7043771 , 19.60673401, 19.50909091, 19.41144781,
        19.31380471, 19.21616162, 19.11851852, 19.02087542, 18.92323232,
        18.82558923, 18.72794613, 18.63030303, 18.53265993, 18.43501684,
        18.33737374, 18.23973064, 18.14208754, 18.04444444, 17.94680135,
        17.84915825, 17.75151515, 17.65387205, 17.55622896, 17.45858586,
        17.36094276, 17.26329966, 17.16565657, 17.06801347, 16.97037037,
        16.87272727, 16.77508418, 16.67744108, 16.57979798, 16.48215488,
        16.38451178, 16.28686869, 16.18922559, 16.09158249, 15.99393939,
        15.8962963 , 15.7986532 , 15.7010101 , 15.603367  , 15.50572391,
        15.40808081, 15.31043771, 15.21279461, 15.11515152, 15.01750842,
        14.91986532, 14.82222222, 14.72457912, 14.6

In [301]:
annual_mb = np.squeeze(annual_mb)
print(np.shape(annual_mb))

(6, 100)


In [305]:
###not properly adapted to plot all six glaciers, was working for only one glacier

# Plot the results
plt.figure(figsize=[14,6])

for i in range(6):
#     gdir = gdirs[i]
    plt(annual_mb, surface_h, label='Mass balance, grad='+ str(mb_gradient))
    # Add ELA, where mass balance = 0:
    # plt.axhline(y=ELA, color='k', linestyle='--', linewidth=0.8, label='Equilibrium line altitude')
    plt.axvline(x=0, color='k', linestyle='--', linewidth=0.8)
    plt.xlabel('Annual mass balance (m yr-1)')
    plt.ylabel('Altitude (m)')
    plt.legend(loc='best');

TypeError: 'module' object is not callable

<Figure size 1008x432 with 0 Axes>